In [ ]:
# 🚀 CREDIT RISK CLASSIFICATION TRAINING - FINAL FULL PIPELINE WITH CONFIG SUPPORT

%pip install scikit-learn pyyaml

import mlflow
import yaml
import numpy as np
import pandas as pd
import warnings
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from mlflow.models.signature import infer_signature
from pyspark.sql import SparkSession

warnings.filterwarnings("ignore")

print("=" * 80)
print("🚀 CREDIT RISK CLASSIFICATION - CONFIG DRIVEN TRAINING")
print("=" * 80)

# -----------------------------------------
# 1️⃣ Load pipeline config (pipeline_config.yml)
# -----------------------------------------
with open("pipeline_config.yml", "r") as f:
    pipeline_cfg = yaml.safe_load(f)

MODEL_TYPE = pipeline_cfg["model"]["type"]
EXPERIMENT_NAME = pipeline_cfg["experiment"]["name"]
MODEL_ARTIFACT_PATH = pipeline_cfg["experiment"]["artifact_path"]
RAW_INPUT_TABLE = pipeline_cfg["data"]["input_table"]
FEATURES = pipeline_cfg["data"]["features"]
LABEL_COL = pipeline_cfg["data"]["label"]

TEST_SIZE = pipeline_cfg["data"]["split"]["test_size"]
RANDOM_STATE = pipeline_cfg["data"]["split"]["random_state"]
STRATIFY = pipeline_cfg["data"]["split"]["stratify"]

print("✅ Pipeline config loaded.")

# -----------------------------------------
# 2️⃣ Load experiment config (config.yml)
# -----------------------------------------
with open("config.yml", "r") as f:
    experiment_cfg = yaml.safe_load(f)

EXPERIMENT_LIST = experiment_cfg["experiments"]
print(f"🔍 Loaded {len(EXPERIMENT_LIST)} experiment variations.")

# -----------------------------------------
# 3️⃣ Load Raw Data (NOT preprocessed)
# -----------------------------------------
spark = SparkSession.builder.appName("CreditRiskTraining").getOrCreate()

print(f"📦 Loading RAW data → {RAW_INPUT_TABLE}")
df = spark.read.table(RAW_INPUT_TABLE).toPandas()

X = df[FEATURES]
y = df[LABEL_COL]

# 🔥 Convert textual labels ("yes", "no") → numeric 1/0
if y.dtype == "object":
    y = y.map({"yes": 1, "no": 0}).astype(int)

# -----------------------------------------
# 4️⃣ Setup Shared Preprocessing Pipeline
# -----------------------------------------
categorical_cols = [col for col in X.columns if X[col].dtype == "object"]
numeric_cols = [col for col in X.columns if col not in categorical_cols]

print(f"🔧 Categorical Features: {len(categorical_cols)}")
print(f"🔧 Numeric Features: {len(numeric_cols)}")

preprocessor = ColumnTransformer(
    transformers=[
        ("categorical", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("numeric", StandardScaler(), numeric_cols)
    ]
)

# -----------------------------------------
# 5️⃣ Train model for each config experiment
# -----------------------------------------
mlflow.set_tracking_uri("databricks")
mlflow.set_registry_uri("databricks-uc")
mlflow.set_experiment(EXPERIMENT_NAME)

stratify_option = y if STRATIFY else None

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, stratify=stratify_option, random_state=RANDOM_STATE
)

results = []

for exp in EXPERIMENT_LIST:
    
    exp_name = exp["name"]
    params = exp["params"]

    print(f"\n⚙️ Training experiment: {exp_name}")

    rf_model = RandomForestClassifier(random_state=RANDOM_STATE, **params)

    full_pipeline = Pipeline([
        ("preprocessing", preprocessor),
        ("model", rf_model)
    ])

    with mlflow.start_run(run_name=exp_name):

        full_pipeline.fit(X_train, y_train)

        y_pred = full_pipeline.predict(X_test)
        y_proba = full_pipeline.predict_proba(X_test)[:, 1]

        # ----- Metrics -----
        metrics = {
            "test_accuracy": accuracy_score(y_test, y_pred),
            "test_precision": precision_score(y_test, y_pred),
            "test_recall": recall_score(y_test, y_pred),
            "test_f1": f1_score(y_test, y_pred),
            "test_roc_auc": roc_auc_score(y_test, y_proba),
        }

        for k, v in metrics.items():
            mlflow.log_metric(k, v)

        signature = infer_signature(X_train, full_pipeline.predict(X_train))

        mlflow.sklearn.log_model(
            sk_model=full_pipeline,
            artifact_path=MODEL_ARTIFACT_PATH,
            signature=signature
        )

        results.append((exp_name, metrics))

print("\n🎉 Training Completed Successfully!")
print("📊 All experiment models logged to MLflow.")

